In [1]:
import joblib

In [4]:
complete_features_matrix = joblib.load('files for cv/Pickles/Sparse_Basic+Temporal+Recency/train.pkl')
complete_features_matrix = complete_features_matrix.tocsr()
complete_features_matrix

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


<8483920x217691 sparse matrix of type '<type 'numpy.float64'>'
	with 154318299 stored elements in Compressed Sparse Row format>

In [5]:
labels = joblib.load('algebra_2005_2008_pickles/labels.compressed')

## Logreg default

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
logreg = LogisticRegression(n_jobs=-1, verbose=2)
scaler = StandardScaler(with_mean=False, copy=False)

In [6]:
clf = Pipeline([('scaler', scaler), ('logreg', logreg)])

In [7]:
clf.fit(complete_features_matrix, labels)

[LibLinear]

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:924: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('scaler', StandardScaler(copy=False, with_mean=False, with_std=True)), ('logreg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=2, warm_start=False))])

In [8]:
joblib.dump(clf, 'algebra_2005_2008_pickles/LogRegdefault_tmprl_rcncy_features.compressed', compress=3)

['algebra_2005_2008_pickles/LogRegdefault_tmprl_rcncy_features.compressed']

## Logreg with standard scaler C10

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [7]:
logreg = LogisticRegression(C=10, n_jobs=-1, verbose=2)
scaler = StandardScaler(with_mean=False, copy=False)

In [8]:
clf = Pipeline([('scaler', scaler), ('logreg', logreg)])

In [9]:
clf.fit(complete_features_matrix, labels)

[LibLinear]

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('scaler', StandardScaler(copy=False, with_mean=False, with_std=True)), ('logreg', LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=2, warm_start=False))])

In [10]:
joblib.dump(clf, 'algebra_2005_2008_pickles/LogRegC10_temporal_features.compressed', compress=3)

['algebra_2005_2008_pickles/LogRegC10_temporal_features.compressed']

## Logreg with standard scaler C1 reg=L1

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
logreg = LogisticRegression(n_jobs=-1, verbose=2, penalty='l1')
scaler = StandardScaler(with_mean=False, copy=False)

In [6]:
clf = Pipeline([('scaler', scaler), ('logreg', logreg)])

In [ ]:
clf.fit(complete_features_matrix, labels)

In [ ]:
joblib.dump(clf, 'algebra_2005_2008_pickles/LogRegl1_temporal_features.compressed', compress=3)

## Testing

In [16]:
test = joblib.load('files for cv/Pickles/Sparse_Basic+Temporal+Recency/test.pkl')

In [17]:
test

<508912x217691 sparse matrix of type '<type 'numpy.float64'>'
	with 9340125 stored elements in COOrdinate format>

In [18]:
probas = clf.predict_proba(test)
probas

array([[ 0.14874206,  0.85125794],
       [ 0.13808814,  0.86191186],
       [ 0.00965918,  0.99034082],
       ..., 
       [ 0.23665212,  0.76334788],
       [ 0.04763265,  0.95236735],
       [ 0.03354199,  0.96645801]])

In [19]:
ones = []
zeros = []
for zero_prob, one_prob in probas:
    if one_prob > zero_prob:
        ones.append(one_prob)
    else:
        zeros.append(one_prob)
        
print len(ones), len(zeros)

484680 24232


In [20]:
import csv
with open('algebra_2008_2009_submissiona.txt', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t')
    spamwriter.writerow(['Row', 'Correct First Attempt'])
    for index, (zero_prob, one_prob) in enumerate(probas):
        spamwriter.writerow([index+1, float(one_prob)])